In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

In [3]:
import math
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

from tqdm.auto import tqdm
from statsmodels.formula.api import ols
from IPython.core.display_functions import display
from utilities.impact_prediction.arima import ArimaModel
from utilities.impact_prediction.multilayer_nn_model import NNModel
from utilities.impact_prediction.linear_regression_model import LRModel
from utilities.utils import read_json, shared_dir, split_dataset, get_cuda_availability, \
    data_exploration_dir, arima_dir, figures_dir

In [4]:
tqdm.pandas()
pd.set_option('display.max_colwidth', None)

logging.basicConfig(level=logging.CRITICAL)

DEVICE = get_cuda_availability()

### Load clusters and display samples

In [6]:
time_limit = 9
space_limit = 10
n_clusters = 10_000

cluster_type = 'custom'
params = [time_limit, space_limit]

In [ ]:
param_file_key = '_'.join([str(x) for x in params])
clusters = {k: pd.DataFrame(v) for k, v in read_json(filename=f'{shared_dir}/{cluster_type}_clusters/labelled/{param_file_key}.json').items()}

print(f'Number of clusters: {len(clusters)}')
for k, v in list(clusters.items())[:5]:
    print(f'Key: {k}')
    display(v.head())

In [ ]:
filename = f'{shared_dir}/{cluster_type}_clusters/formatted/{param_file_key}.json'
formatted_clusters = pd.DataFrame(list(read_json(filename=filename).values()))
formatted_clusters.head()

### Graph cluster feature distributions

In [ ]:
fig = px.histogram(formatted_clusters, x='time_of_day', template='plotly')
fig.write_json(f'{data_exploration_dir}/time_of_day_hist.json')
fig.write_html(f'{data_exploration_dir}/time_of_day_hist.html')
fig.show()

In [ ]:
fig = px.histogram(formatted_clusters, x='weekday', template='plotly')
fig.write_json(f'{data_exploration_dir}/weekday_hist.json')
fig.write_json(f'{data_exploration_dir}/weekday_hist.html')
fig.show()

In [ ]:
formatted_clusters['grouped'] = formatted_clusters.apply(lambda x: str((x['weekday'], x['time_of_day'])), axis=1)

fig = px.histogram(formatted_clusters, x='grouped', template='plotly')
fig.update_xaxes(categoryorder='category ascending')
fig.write_json(f'{data_exploration_dir}/grouped_hist.json')
fig.write_html(f'{data_exploration_dir}/grouped_hist.html')
fig.show()

In [26]:
groups = formatted_clusters.groupby('grouped')

grouped_xs = []
grouped_ys = []
tod_xs = []
tod_ys = []
dow_xs = []
dow_ys = []

for label, group in groups:
    grouped_xs.append(label)
    grouped_ys.append(group['size'].mean())

groups = formatted_clusters.groupby('time_of_day')

for label, group in groups:
    tod_xs.append(label)
    tod_ys.append(group['size'].mean())

groups = formatted_clusters.groupby('weekday')

for label, group in groups:
    dow_xs.append(label)
    dow_ys.append(group['size'].mean())

In [ ]:
size_df = pd.DataFrame({'label': grouped_xs, 'size': grouped_ys})
fig = px.bar(size_df, x='label', y='size')
fig.update_layout(
    template='plotly',
    xaxis_title="Group - (dow, tod)",
    yaxis_title="Avg. Size",
)
fig.write_json(f'{data_exploration_dir}/size_hist.json')
fig.write_html(f'{data_exploration_dir}/size_hist.html')
fig.show()

In [ ]:
size_df = pd.DataFrame({'label': tod_xs, 'size': tod_ys})
fig = px.bar(size_df, x='label', y='size')
fig.update_layout(
    template='plotly',
    xaxis_title="Time of Day",
    yaxis_title="Avg. Size",
)
fig.write_json(f'{data_exploration_dir}/tod_size_hist.json')
fig.write_html(f'{data_exploration_dir}/tod_size_hist.html')
fig.show()

In [ ]:
size_df = pd.DataFrame({'label': dow_xs, 'size': dow_ys})
fig = px.bar(size_df, x='label', y='size')
fig.update_layout(
    template='plotly',
    xaxis_title="Day of the Week",
    yaxis_title="Avg. Size",
)
fig.write_json(f'{data_exploration_dir}/dow_size_hist.json')
fig.write_html(f'{data_exploration_dir}/dow_size_hist.html')
fig.show()

## 2-Way Anova Test

In [ ]:
model = ols('size ~ C(weekday) + C(time_of_day) + C(weekday):C(time_of_day)', data=formatted_clusters).fit()
print(model.summary())

anova = sm.stats.anova_lm(model, typ=2)
anova.to_pickle(f'{figures_dir}/visualisation/anova.pickle')
anova.head()

## Build training, validation and testing datasets

Split is 70% training, 20% validation and 10% testing

In [ ]:
trdf, tvdf = split_dataset(formatted_clusters, split=0.7)
tvdf, tedf = split_dataset(tvdf, split=0.6666666)

print(f'Training Size: {len(trdf)} - {(len(trdf) / len(formatted_clusters)) * 100: .2f}%')
print(f'Validation Size: {len(tvdf)} - {(len(tvdf) / len(formatted_clusters)) * 100: .2f}%')
print(f'Testing Size: {len(tedf)} - {(len(tedf) / len(formatted_clusters)) * 100: .2f}%')

## Linear Regression Model

### Generate baselines

In [ ]:
vs = {label: group['size'].mean() for label, group in trdf.groupby(['weekday', 'time_of_day'])}

tvdf['prediction'] = tvdf.apply(lambda x: vs.get((x['weekday'], x['time_of_day']), 0), axis=1)
tedf['prediction'] = tedf.apply(lambda x: vs.get((x['weekday'], x['time_of_day']), 0), axis=1)
display(tvdf.head())

mse_val = np.mean((np.array(tvdf['size'].tolist()) - np.array(tvdf['prediction'].tolist())) ** 2)
mse_test = np.mean((np.array(tedf['size'].tolist()) - np.array(tedf['prediction'].tolist())) ** 2)

print(mse_val)
print(mse_test)

### Test weekday variable

In [ ]:
averages = {label * 6: group['size'].mean() for label, group in trdf.groupby(['weekday'])}
model = LRModel(trdf, tvdf, tedf, averages, cols=['weekday'], label=cluster_type)

model.train()
model.eval()
model.test()

### Test time of day variable

In [ ]:
averages = {label * 23: group['size'].mean() for label, group in trdf.groupby(['time_of_day'])}
model = LRModel(trdf, tvdf, tedf, averages, cols=['time_of_day'], label=cluster_type)

model.train()
model.eval()
model.test()

### Test both variables

In [ ]:
averages = {label: group['size'].mean() for label, group in trdf.groupby(['weekday', 'time_of_day'])}
model = LRModel(trdf, tvdf, tedf, averages, cols=['weekday', 'time_of_day'], label=cluster_type)

model.train()
model.eval()
model.test()

## Neural Network

### Set hyperparameters

In [47]:
epochs = 20
batch_size = 64
learning_rate = 0.001

### Build and train model

In [ ]:
nn_model = NNModel(trdf[['size', 'weekday', 'time_of_day']].copy(), batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, device=DEVICE)
nn_model.train()

### Validate model

In [ ]:
vs_prime = nn_model.validate(tvdf)

### Test model

In [ ]:
nn_model.test(tedf)

## ARIMA Model

### Format and visualise dataset for use by ARIMA model

In [ ]:
arima_df = formatted_clusters.sort_values(by='local_time')
arima_df['local_time'] = arima_df['local_time'].apply(lambda x: x[:-6])
arima_df.set_index('local_time', drop=False, inplace=True)
arima_df.head()

In [ ]:
fig = px.scatter(formatted_clusters, x='local_time', y='size', template='plotly')
fig.write_json(f'{arima_dir}/arima_size_local_time_graph.json')
fig.write_html(f'{arima_dir}/arima_size_local_time_graph.html')
fig.show()

### Create ARIMA model for all data

In [ ]:
split_point = math.floor(len(arima_df) * 0.8)
training = arima_df.head(split_point).copy()
validation = arima_df.tail(len(arima_df) - split_point).copy()

arima_model = ArimaModel(data=training)

### Tune model

In [ ]:
arima_model.tune(training, validation, ds=[], ps=[0, 1, 2, 3], qs=[0, 1, 2, 3, 4], cluster_type=cluster_type, do_grid_search=False)

### Run model

In [ ]:
arima_model.run(training, validation, filename=f'arima_all_1_0_4')

### Experiment on London-only data

London Bounding Box --- [-0.489, 51.28, 0.236, 51.686]

In [ ]:
data = arima_df[(arima_df.x < 51.686) & (arima_df.x > 51.28) & (arima_df.y < 0.236) & (arima_df.y > -0.489)].copy()
split_point = math.floor(len(data) * 0.8)
training = data.head(split_point).copy()
validation = data.tail(len(data) - split_point).copy()

arima_model = ArimaModel(data=training)

In [ ]:
arima_model.run(training, validation, filename=f'arima_london_1_0_4')